# Mouse Population Density Estimation

Here I'm playing around with creating a small neural network to take in the results of the simualtions and try to predict the true population density.

## Notes

I've tried using just the density estimations of each square and it didn't do much better than a linear model (somewhat unsurprisingly). I'm **now incorperating trap spacing and catch radius**. 

## To-Do

- Use some sort of inverse weighting scheme because most of the estimates are already good because the model works well. We need to emphasize the areas where the method doesn't work as well to beat a linear model.
- Try adding in other metrics (variance of the error, for instance) and maybe even make it the target for training rather than L2 error.

In [ ]:
import pandas as pd
import mxnet as mx
from mxnet import nd, autograd, gluon

In [ ]:
data_ctx = mx.gpu()
model_ctx = mx.gpu()

In [ ]:
simdata = pd.read_csv("data/trainingdata.csv")
simdata.shape

In [ ]:
batch_size = 100000

# num_train_samples = int(simdata.shape[0]*.08)
# num_test_samples = int(simdata.shape[0]*.01)
num_train_samples = 100*batch_size
num_test_samples = 25*batch_size

print("Num batches: train =", num_train_samples/batch_size, "test =", num_test_samples/batch_size)

In [ ]:
simtrain = simdata.sample(n=num_train_samples)
simtest = simdata.drop(simtrain.index).sample(num_test_samples)

predictors = ["square1", "square2","square3", "square4","square5", "square6","square7", "square8", "TrapSpacing", "CatchRadius"]

Xtrain = nd.array(simtrain[predictors], ctx=data_ctx)
Ytrain = nd.array(simtrain["Density"], ctx=data_ctx)

Xtest = nd.array(simtest[predictors], ctx=data_ctx)
Ytest = nd.array(simtest["Density"], ctx=data_ctx)

train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(Xtrain, Ytrain), batch_size=batch_size, shuffle=True)
test_data = gluon.data.DataLoader(gluon.data.ArrayDataset(Xtest, Ytest), batch_size=batch_size, shuffle=False)

In [ ]:
net = gluon.nn.Sequential()

with net.name_scope():
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(8, activation="relu"))
    net.add(gluon.nn.Dense(1))
    net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=model_ctx)

In [ ]:
net

In [ ]:
square_loss = gluon.loss.L2Loss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.00001})

In [ ]:
loss_train_sequence = []
loss_test_sequence = []

In [ ]:
epochs = 1000

for e in range(epochs):
    # train epoch loop
    cumulative_train_loss = 0
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        cumulative_train_loss += nd.sum(loss).asscalar()
    loss_train_sequence.append(cumulative_train_loss/num_train_samples)
    
#     print("Epoch %s, train loss: %s" % (e, cumulative_train_loss/num_train_samples))

    # test epoch loss loop
    cumulative_test_loss = 0
    for i, (data, label) in enumerate(test_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        output = net(data)
        loss = square_loss(output, label)
        cumulative_test_loss += nd.sum(loss).asscalar()
    loss_test_sequence.append(cumulative_test_loss/num_test_samples)
    
    print("Epoch %s, train loss: %s, test loss: %s" % (e, cumulative_train_loss/num_train_samples, cumulative_test_loss/num_test_samples))
    if e % 10 is 0:
        print("Test values:", Ytest[0:5].asnumpy())
        print("Predicted values:", net(Xtest)[0:5,0].asnumpy())

In [ ]:
# plot the convergence of the estimated loss function
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

plt.figure(num=None,figsize=(8, 6))
plt.plot(loss_train_sequence[10:])
plt.plot(loss_test_sequence[10:])

# Adding some bells and whistles to the plot
plt.grid(True, which="both")
plt.xlabel('epoch',fontsize=14)
plt.ylabel('average loss',fontsize=14)

In [ ]:
net.summary(nd.random.uniform(shape=(8,10), ctx=data_ctx))

In [ ]:
square_loss(net(Xtrain), Ytrain).mean()

In [ ]:
square_loss(net(Xtest), Ytest).mean()

In [ ]:
plt.hist((Ytest.flatten()-net(Xtest)).asnumpy())

In [ ]:
Ytest

In [ ]:
net(Xtest)